# User Defined Network

## Includes

In [ ]:
# mass includes
import torch as t
from ipynb.fs.full.module import BasicModule

 ## MobileNetV2

In [ ]:
class dpConv2d(BasicModule):
    def __init__(self, in_channels, out_channels, stride, padding):
        super(dpConv2d, self).__init__()
        self.features = t.nn.Sequential(
            t.nn.Conv2d(
                in_channels,
                in_channels,
                3,
                stride=stride,
                padding=padding,
                groups=in_channels,
                bias=False), t.nn.BatchNorm2d(in_channels),
            t.nn.ReLU(inplace=True),
            t.nn.Conv2d(in_channels, out_channels, 1, bias=False),
            t.nn.BatchNorm2d(out_channels))

    def forward(self, x):

        return self.features(x)


class invertedResidual(BasicModule):
    def __init__(self, in_channels, out_channels, stride):
        super(invertedResidual, self).__init__()

        inter_channels = in_channels * 6
        self.features = t.nn.Sequential(
            t.nn.Conv2d(in_channels, inter_channels, 1, bias=False),
            t.nn.BatchNorm2d(inter_channels), t.nn.ReLU(inplace=True),
            dpConv2d(inter_channels, out_channels, stride, 1))
        if stride == 1 and in_channels == out_channels:
            self.add_residual = True
        else:
            self.add_residual = False

    def forward(self, x):
        if self.add_residual:
            return x + self.features(x)
        else:
            return self.features(x)


class bottleNeck(BasicModule):
    def __init__(self, in_channels, out_channels, stride, repeat):
        super(bottleNeck, self).__init__()

        layers = []
        for index in range(0, repeat):
            if index == 0:
                layers.append(
                    invertedResidual(in_channels, out_channels, stride))
            else:
                layers.append(invertedResidual(out_channels, out_channels, 1))
        self.features = t.nn.Sequential(*layers)

    def forward(self, x):

        return self.features(x)


class MobileNetV2(BasicModule):
    def __init__(self, train=True):
        super(MobileNetV2, self).__init__()
        self.model_name = 'MobileNetV2'

        # feature extraction layers
        self.features = t.nn.Sequential(
            # conv 1
            t.nn.Conv2d(3, 32, 3, stride=2, padding=1, bias=False),
            t.nn.BatchNorm2d(32),
            t.nn.ReLU(inplace=True),
            # block 2
            dpConv2d(32, 16, 1, 1),
            # block 3
            bottleNeck(16, 24, 2, 2),
            # block 4
            bottleNeck(24, 32, 2, 3),
            # block 5
            bottleNeck(32, 64, 2, 4),
            # block 6
            bottleNeck(64, 96, 1, 3),
            # block 7
            bottleNeck(96, 160, 2, 3),
            # block 8
            bottleNeck(160, 320, 1, 1))

        # classification layers
        if train:
            self.classifier_P1 = t.nn.Sequential(
                # conv 9 (with dropout)
                t.nn.Conv2d(320, 1280, 1, bias=False),
                t.nn.BatchNorm2d(1280),
                t.nn.ReLU(inplace=True),
                t.nn.AvgPool2d(7, stride=1),
                t.nn.Dropout(0.2))
        else:
            self.classifier_P1 = t.nn.Sequential(
                # conv 9 (without dropout)
                t.nn.Conv2d(320, 1280, 1, bias=False),
                t.nn.BatchNorm2d(1280),
                t.nn.ReLU(inplace=True),
                t.nn.AvgPool2d(7, stride=1))

        self.classifier_P2 = t.nn.Linear(1280, 1000)

        # initialization
        self.initLayers()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier_P1(x)
        x = x.view(x.size(0), -1)
        x = self.classifier_P2(x)
        x = t.nn.functional.softmax(x, dim=1)

        return x